# Import everything


In [1]:
from typing import Tuple
from os.path import exists
import torch
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from settings import cfg
from helper import loader
import export_result


# Load everything

In [2]:
config = cfg.TrainConfig()


Resuming from /home/phuonghd/NHAT/BallPunchAIModel/results/vgg11_bn_pr_state_dict.pt
Param size: 491.257MB
Buffer size: 0.021MB
Stats:
        | Number of not-punching: 34006 	| Number of punching: 6355


# Train

In [3]:
from sklearn.metrics import f1_score


def train(train_idx: np.ndarray) -> 'Tuple(float, float)':
    config.model.train()
    train_dataloader = config.get_dataloader(train_idx)
    total_loss_train = 0

    prediction_array = []
    label_array = []
    for image, label in tqdm(train_dataloader):
        image = image.to(config.device, dtype=torch.float)
        image = config.transforms(image)
        if config.use_grayscale:
            image = config.grayscale(image)
        output = config.model(image)

        label = label.to(config.device, dtype=torch.uint8)
        batch_loss = config.criterion(output, label)
        total_loss_train += batch_loss.item()

        prediction = output.argmax(dim=1)

        prediction_array.append(prediction.cpu().numpy())
        label_array.append(label.cpu().numpy())

        config.optimizer.zero_grad()
        batch_loss.backward()
        config.optimizer.step()
        config.model.zero_grad()

    prediction_array = np.concatenate(prediction_array)
    label_array = np.concatenate(label_array)

    total_accuracy_train = (prediction_array == label_array).sum().item()
    f1_score_train = f1_score(label_array, prediction_array, average='macro')

    return (total_loss_train/train_idx.shape[0],
        total_accuracy_train/train_idx.shape[0],
        f1_score_train)


In [4]:
def judge(judge_idx: np.ndarray) -> 'Tuple(float, float)':
    config.model.eval()
    judge_dataloader = config.get_dataloader(judge_idx)
    total_loss_judge = 0

    prediction_array = []
    label_array = []
    with torch.no_grad():
        for image, label in tqdm(judge_dataloader):
            image = image.to(config.device, dtype=torch.float)
            if config.use_grayscale:
                image = config.grayscale(image)
            output = config.model(image)
            label = label.to(config.device, dtype=torch.uint8)

            batch_loss = config.criterion(output, label)
            total_loss_judge += batch_loss.item()

            prediction = output.argmax(dim=1)
            prediction_array.append(prediction.cpu().numpy())
            label_array.append(label.cpu().numpy())

    prediction_array = np.concatenate(prediction_array)
    label_array = np.concatenate(label_array)

    total_accuracy_judge = (prediction_array == label_array).sum().item()
    f1_score_judge = f1_score(label_array, prediction_array, average='macro')

    return (total_loss_judge/judge_idx.shape[0],
        total_accuracy_judge/judge_idx.shape[0],
        f1_score_judge)

In [5]:
min_loss_judge = float('inf')
last_loss_judge = float('inf')
des_sequence = 0
under_min = 0
last_submit = 0

for epoch, (train_idx, judge_idx) in enumerate(config.get_split()):
    print(f'''Starting epoch {epoch+1}
    | Train size:     {train_idx.shape[0]}   | Judge size:     {judge_idx.shape[0]}''')
    avg_loss_train, avg_accu_train, f1_score_train = train(train_idx)
    avg_loss_judge, avg_accu_judge, f1_score_judge = judge(judge_idx)

    print(
        f'''Epoch: {epoch+1} 
    | Train Loss:     {avg_loss_train:.3f}   | Judge Loss:     {avg_loss_judge:.3f}
    | Train Accuracy: {avg_accu_train:.3f}   | Judge Accuracy: {avg_accu_judge:.3f}
    | Train F1 Score: {f1_score_train:.3f}   | Judge F1 Score: {f1_score_judge:.3f}''')

    if last_loss_judge < avg_loss_judge:
        des_sequence += 1
    if min_loss_judge < avg_loss_judge:
        under_min += 1
    else:
        config.save_checkpoint()
        print(f'''Judge loss improved:
    | From:           {min_loss_judge:.3f}   | To: {avg_loss_judge:.3f}''')
        min_loss_judge = avg_loss_judge
        under_min = des_sequence = 0

    if under_min >= cfg.early_stop:
        print(f"Early stop. Not better than best for {under_min} epochs.")
        config.load_best()
        print(f"Best model loaded.")
        des_sequence = under_min = 0
    elif des_sequence >= cfg.des_sequence_early_stop:
        print(f"Early stop. Not improved for {des_sequence} epochs.")
        config.load_best()
        print(f"Best model loaded.")
        des_sequence = under_min = 0

    if last_submit == 0:
        last_submit = f1_score_judge
        print("Initial submit")
        export_result.submit(config)
    elif f1_score_judge - last_submit > 0.05:
        print(f'''Submitting:
    | F1 Score: {f1_score_judge:.3f} | Last Submit: {last_submit:.3f}''')
        last_submit = f1_score_judge
        export_result.submit(config)
    else:
        print(f'''Not submitting:
    | F1 Score: {f1_score_judge:.3f} | Last Submit: {last_submit:.3f}''')
    print("\n____________________________________________")


Starting epoch 1
    | Train size:     36324   | Judge size:     4037


  0%|          | 0/1136 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

Epoch: 1 
    | Train Loss:     0.022   | Judge Loss:     0.022
    | Train Accuracy: 0.752   | Judge Accuracy: 0.848
    | Train F1 Score: 0.497   | Judge F1 Score: 0.459
Judge loss improved:
    | From:           inf   | To: 0.022
Initial submit
Starting evaluation


  0%|          | 0/296 [00:00<?, ?it/s]

...

Epoch: 27 
    | Train Loss:     0.022   | Judge Loss:     0.022
    | Train Accuracy: 0.797   | Judge Accuracy: 0.835
    | Train F1 Score: 0.486   | Judge F1 Score: 0.455
Not submitting:
    | F1 Score: 0.455 | Last Submit: 0.459

____________________________________________
Starting epoch 28
    | Train size:     36325   | Judge size:     4036


  0%|          | 0/1136 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

Epoch: 28 
    | Train Loss:     0.022   | Judge Loss:     0.022
    | Train Accuracy: 0.747   | Judge Accuracy: 0.835
    | Train F1 Score: 0.500   | Judge F1 Score: 0.455
Not submitting:
    | F1 Score: 0.455 | Last Submit: 0.459

____________________________________________
Starting epoch 29
    | Train size:     36325   | Judge size:     4036


  0%|          | 0/1136 [00:00<?, ?it/s]